In [1]:
import sys
import json
import re
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Tuple
import torch
import subprocess
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

## Number of few shot examples = 3 / 5 / 10

In [2]:
FINE_LABELS = [
        "Ad-hominem", "Appeal-to-authority", "Appeal-to-emotion", 
        "Causal-oversimplification", "Cherry-picking", "Circular-reasoning", 
        "Doubt", "Evading-the-burden-of-proof", "False-analogy", 
        "False-dilemma", "Flag-waving", "Hasty-generalization", 
        "Loaded-language", "Name-calling-or-labelling", "Red-herring", 
        "Slippery-slope", "Slogan", "Strawman", "Thought-terminating-cliches", 
        "Vagueness"
    ]

In [3]:
Model_Name = "Qwen/Qwen2.5-3B-Instruct"
# Model_Name = "Qwen/Qwen2.5-7B-Instruct"
# Model_Name = "mistralai/Mistral-7B-Instruct-v0.3"
# Model_Name = "google/gemma-3n-E2B-it"
# Model_Name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# Model_Name = "microsoft/Phi-3-mini-4k-instruct"
# Model_Name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

In [4]:
def init_model(model_name=Model_Name, use_cpu=False, use_4bit=False):
    device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
    print(f"Loading model on device: {device}")

    if use_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer, device

In [5]:
def format_for_model(prompt: str, tokenizer) -> str:
    messages = [{"role": "user", "content": prompt}]
    if hasattr(tokenizer, "apply_chat_template"):
        try:
            return tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        except Exception:
            return prompt
    return prompt

In [6]:
def generate_text(prompt: str, model, tokenizer, device, max_tokens: int = 512, temperature: float = 0.0) -> str:
    text = format_for_model(prompt, tokenizer)
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True if temperature > 0 else False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )

    generated = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)

    del inputs, outputs, generated
    torch.cuda.empty_cache()

    return response

In [7]:
def extract_labels_from_response(response: str) -> list:
    labels = []
    try:
        start = response.find("{")
        end = response.rfind("}") + 1
        if start != -1 and end > start:
            data = json.loads(response[start:end])
            labels = data.get("labels") or data.get("fallacies") or []
    except:
        pass

    if not labels:
        try:
            start = response.find("[")
            end = response.rfind("]") + 1
            if start != -1 and end > start:
                labels = json.loads(response[start:end])
        except:
            pass

    if not labels:
        for label in FINE_LABELS:
            if label in response:
                labels.append(label)

    valid_labels = []
    for label in labels:
        label = label.strip()
        if label in FINE_LABELS:
            valid_labels.append(label)
        else:
            normalized = label.replace(" ", "-")
            if normalized in FINE_LABELS:
                valid_labels.append(normalized)

    return list(set(valid_labels))

In [8]:
def build_few_shot_examples(train_df: pd.DataFrame, n_examples_per_label: int = 3,
                            labels_col: str = "labels_a1", max_text_length: int = 300) -> dict:
    examples = defaultdict(list)
    train_df_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

    for idx, row in train_df_shuffled.iterrows():
        labels = str(row[labels_col]).split('|')
        labels = [l.strip() for l in labels if l.strip()]

        text = row['post_text']
        if len(text) > max_text_length:
            text = text[:max_text_length] + "..."

        for label in labels:
            if label in FINE_LABELS and len(examples[label]) < n_examples_per_label:
                examples[label].append(text)
        if all(len(examples[label]) >= n_examples_per_label for label in FINE_LABELS):
            break

    print(f"\nBuilt few-shot examples for {len(examples)} labels:")
    missing_labels = [label for label in FINE_LABELS if label not in examples]

    for label in FINE_LABELS:
        if label in examples:
            print(f"   {label}: {len(examples[label])} examples")
        else:
            print(f"   {label}: 0 examples (NOT FOUND IN TRAINING DATA)")

    if missing_labels:
        print(f"\n  WARNING: {len(missing_labels)} labels have no examples!")
        print(f"Missing: {', '.join(missing_labels)}")

    return dict(examples)

In [9]:
def create_few_shot_prompt(text: str, examples: dict, max_examples: int = 3) -> str:
    prompt = """You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting only favorable evidence
- Circular-reasoning: Conclusion is assumed in the premise
- Doubt: Questioning credibility without evidence
- Evading-the-burden-of-proof: Avoiding providing evidence
- False-analogy: Comparing incomparable things
- False-dilemma: Presenting only two options when more exist
- Flag-waving: Appealing to nationalism/patriotism
- Hasty-generalization: Drawing conclusions from insufficient evidence
- Loaded-language: Using emotionally charged words
- Name-calling-or-labelling: Using derogatory labels
- Red-herring: Introducing irrelevant information
- Slippery-slope: Claiming one thing will lead to extreme consequences
- Slogan: Using catchy phrases instead of arguments
- Strawman: Misrepresenting opponent's argument
- Thought-terminating-cliches: Using clichés to stop critical thinking
- Vagueness: Being intentionally unclear

"""
    prompt += "EXAMPLES FROM TRAINING DATA:\n\n"
    example_posts = []
    for label, texts in examples.items():
        if texts:
            example_posts.append((texts[0], label))

    import random
    if len(example_posts) > max_examples:
        random.seed(42)
        example_posts = random.sample(example_posts, max_examples)

    for idx, (example_text, label) in enumerate(example_posts, 1):
        prompt += f"Example {idx}:\n"
        prompt += f"Text: \"{example_text}\"\n"
        prompt += f"Contains: {label}\n\n"

    prompt += f"""Now analyze this NEW text and identify ALL fallacy types present in that text:

TEXT: "{text}"

IMPORTANT INSTRUCTIONS:
1. Identify ALL fallacies present. There can be multiple fallacies
2. Only use labels from the above list of available fallacy types. don't create fallacies on your own
3. If no fallacies found, return empty array

Output format: Return ONLY a JSON array of labels, like this:
["Label1", "Label2", "Label3"]

If no fallacies: []

Your response (JSON array only):"""

    return prompt

In [10]:
def predict_labels(text: str, examples: dict, model, tokenizer, device) -> list:
    prompt = create_few_shot_prompt(text, examples)
    response = generate_text(prompt, model, tokenizer, device, max_tokens=256, temperature=0.0)

    print(f"\nPROMPT (first 500 chars):\n{prompt[:500]}")
    print(f"\nRESPONSE:\n{response}\n")

    labels = extract_labels_from_response(response)
    return sorted(labels)

In [11]:
def predict_batch(df: pd.DataFrame, examples: dict, model, tokenizer, device) -> pd.DataFrame:
    predictions = []

    for idx, row in df.iterrows():
        print(f"\nProcessing post {idx + 1}/{len(df)}: {row['post_id']}")

        text = row['post_text']
        predicted_labels = predict_labels(text, examples, model, tokenizer, device)
        labels_str = "|".join(predicted_labels) if predicted_labels else ""
        predictions.append({
            'post_id': row['post_id'],
            'post_date': row.get('post_date', ''),
            'post_topic_keywords': row.get('post_topic_keywords', ''),
            'post_text': text,
            'labels_a1': labels_str,
            'labels_a2': labels_str
        })

        print(f"Predicted labels: {predicted_labels}")

    return pd.DataFrame(predictions)

In [12]:
def save_predictions(predictions_df: pd.DataFrame, output_path: str):
    predictions_df.to_csv(output_path, sep='\t', index=False)
    print(f"\nPredictions saved to: {output_path}")

In [13]:
def evaluate(gold_path: str, pred_path: str):
    cmd = [
        sys.executable, "eval.py",
        "-S", "A",
        "-G", gold_path,
        "-P", pred_path
    ]

    print("\n" + "="*60)
    print("RUNNING EVALUATION")
    print("="*60)

    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)

In [14]:
if __name__ == "__main__":
    DATA_PATH = "../data/faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-a/train-dev.tsv"
    safe_model_name = Model_Name.replace("/", "_")
    PRED_PATH = f"../results/subtask-a/{safe_model_name}_predicted.tsv"
    GOLD_PATH = f"../results/subtask-a/{safe_model_name}_gold.tsv"

    print("Loading data...")
    df = pd.read_csv(DATA_PATH, sep='\t')
    print(f"Total posts: {len(full_df)}")

    train_df, test_df = train_test_split(df, test_size=0.2,  random_state=42,  shuffle=True)

    print(f"Train: {len(train_df)} posts ({len(train_df)/len(full_df)*100:.1f}%)")
    print(f"Test: {len(test_df)} posts ({len(test_df)/len(full_df)*100:.1f}%)")
    
    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    print("\nInitializing model...")
    model, tokenizer, device = init_model(use_4bit=True)

    print("\nBuilding few-shot examples from training data...")
    examples = build_few_shot_examples(
        train_df,  
        n_examples_per_label=3, 
        labels_col="labels_a1"
    )

    print("\nMaking predictions on test set...")
    predictions_df = predict_batch(test_df, examples, model, tokenizer, device)

    save_predictions(predictions_df, PRED_PATH)

    gold_df = test_df.copy()
    if 'labels_a2' not in gold_df.columns or gold_df['labels_a2'].isna().all():
        gold_df['labels_a2'] = gold_df['labels_a1']
    gold_df.to_csv(GOLD_PATH, sep='\t', index=False)
    print(f"Gold labels saved to: {GOLD_PATH}")

    print("\nEvaluating predictions...")
    evaluate(GOLD_PATH, PRED_PATH)

Loading data...
Total posts: 1152
Train: 921 posts (79.9%)
Test: 231 posts (20.1%)

Label distribution in test set:
  Appeal-to-emotion: 110
  Loaded-language: 97
  Vagueness: 63
  Name-calling-or-labelling: 56
  Hasty-generalization: 44
  Doubt: 40
  nan: 39
  Evading-the-burden-of-proof: 31
  Thought-terminating-cliches: 24
  Slogan: 23

Initializing model...
Loading model on device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Building few-shot examples from training data...

Built few-shot examples for 20 labels:
   Ad-hominem: 3 examples
   Appeal-to-authority: 3 examples
   Appeal-to-emotion: 3 examples
   Causal-oversimplification: 3 examples
   Cherry-picking: 3 examples
   Circular-reasoning: 3 examples
   Doubt: 3 examples
   Evading-the-burden-of-proof: 3 examples
   False-analogy: 3 examples
   False-dilemma: 3 examples
   Flag-waving: 3 examples
   Hasty-generalization: 3 examples
   Loaded-language: 3 examples
   Name-calling-or-labelling: 3 examples
   Red-herring: 3 examples
   Slippery-slope: 3 examples
   Slogan: 3 examples
   Strawman: 3 examples
   Thought-terminating-cliches: 3 examples
   Vagueness: 3 examples

Making predictions on test set...

Processing post 1/231: 353

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple falla


PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
[]

Predicted labels: []

Processing post 14/231: 745

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions in

...


PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
["Thought-terminating-cliches"]

Predicted labels: ['Thought-terminating-cliches']

Processing post 166/231: 1099

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authori


PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
["Hasty-generalization"]

Predicted labels: ['Hasty-generalization']

Processing post 180/231: 128

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence



PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
["Ad-hominem"]

Predicted labels: ['Ad-hominem']

Processing post 194/231: 416

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion:


PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
[]

Predicted labels: []

Processing post 208/231: 536

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions i


PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions instead of using logic
- Causal-oversimplification: Oversimplifying cause-effect relationships
- Cherry-picking: Selecting

RESPONSE:
[]

Predicted labels: []

Processing post 222/231: 1017

PROMPT (first 500 chars):
You are an expert in identifying logical fallacies and propaganda techniques in text.

Your task is to identify which fallacy types are present in the given text. A text can have multiple fallacies.

AVAILABLE FALLACY TYPES:
- Ad-hominem: Attacking the person instead of the argument
- Appeal-to-authority: Using authority as evidence
- Appeal-to-emotion: Manipulating emotions 